# XML Springboard mondial exercise
****
+ reading data from world bank sample XML file
+ utilize xml.etree to read data from xml   
+ utilize numpy to read data in data frame   
+ undestand the data from file  
+ work on exercise to be completed and submit
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

## Import the libararies


In [2]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

****
## Load world bank XML  file
+ read XML file using xml.etree 

In [3]:
# document tree is having XMl data from file
document_tree = ET.parse( './data/mondial_database.xml' )

****
## XML exercise

The questionUsing data in file 'data/mondial_database.xml' and the techniques demonstrated above,
1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

1. Find the 10 countries with the lowest infant mortality rates

Approach 
1. Already have data from the file. 
2. iterate over the country node and build the data in pandas series
3. The data is extracted using method find('name'). And country name is in text property
4. Once data is in Series , i found that sort not working on infant mortality as it is not int or float
5. The getiterator help in handling issue where if tag doesnt exist the error occured
6. The data is converted to float as it is decimal type and sorted and filtered


In [26]:
# 10 countries with lowest mortality
# created empty series and dictonary
coun_series = pd.Series()
dic_coun={}
# loop through all the country tags
for element in document_tree.iterfind('country'):
    capitals_string = ''
    # loop through all the infan mortality tags
    for subelement in element.getiterator('infant_mortality'):
        capitals_string = subelement.text 
        # add to dictonary
        dic_coun[element.find('name').text]=capitals_string

# create series from dictonary
coun_series= pd.Series(dic_coun)
# coverting infant mortality to float & sorting in ascending t get lowest first
coun_series=coun_series.apply(np.float).sort_values(ascending=True)
# getting only top 10
coun_series.head(10)


Monaco            1.81
Japan             2.13
Norway            2.48
Bermuda           2.48
Singapore         2.53
Sweden            2.60
Czech Republic    2.63
Hong Kong         2.73
Macao             3.13
Iceland           3.15
dtype: float64

2. Find the  10 cities with the largest population
3. Find 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

Approach 
1. Already have data from the file. 
2. iterate over the country node and build the data in pandas series
3. Created methods for reusablity & avoidindg exception
4. Extracted data from country node using itertaor, find node.attrib
5. Built data frame from further processing using pandas. 
6. The data is converted to float as it is decimal type and sorted and filtered


In [27]:
# method to chek if the element exist in the node using find
def is_element_in_xml(element,child_element):
    if element.find(child_element):
        return True
    else:
        return False
    

In [28]:
# method to check if attrib exist in the node.
def is_attrib_in_element(element,attrib_name):
    if attrib_name in element.attrib:
        return True
    else:
        return False


In [29]:
# method use to cast string to int
def cast_element_int(val):
    return_val=0
    try:
        return_val=int(val)
    except:
        return_val=np.NAN
    return return_val


In [30]:
# method use to cast string to float
def cast_element_float(val):
    return_val=0
    try:
        return_val=float(val)
    except:
        return_val=np.NAN
    return return_val


In [31]:
#This method use to send the dictonry entry per country ,gives last year population available.
#This method is called and gets added in dataframe
# This method can be passed country as a param or find out from node if empty. This gives flexbility to resue fro both Q 2 & 3 
def find_city_population(ielement,country=''):
        for subelement in ielement.iterfind('city'):
            val_year =1900
            val_population=0
            for pinfo in subelement.iterfind('population'):
                if is_attrib_in_element(pinfo,'year'):
                    t_year=cast_element_int(pinfo.attrib['year'])
                    # this comparison allowed to get the max year and its popultaion
                    if(t_year>val_year):
                        val_year=t_year
                        val_population=cast_element_int(pinfo.text)

            return{
                'country':ielement.find('name').text if(country=='') else country,
                #'countrycode':ielement.attrib['car_code']if(is_attrib_in_element(ielement,'car_code')) else '',
                'city':subelement.find('name').text,
                'year':val_year,
                'population':val_population
           }
            

In [32]:
#This method use to send the dictonry entry per country ,gives find all ethincity data for country.
#This method is called and gets added in dataframe
def find_ethinic_population(ielement):
        val_population=0
        val_year =1900

        for subelement in ielement.iterfind('population'):
            if is_attrib_in_element(subelement,'year'):
                t_year=cast_element_int(subelement.attrib['year'])
                if(t_year>val_year):
                    val_year=t_year
                    val_population=cast_element_float(subelement.text)
            
        for subelement in ielement.iterfind('ethnicgroup'):
            val_ethincity_percent =0
            val_ethincity_name=''
            if is_attrib_in_element(subelement,'percentage'):
                val_ethincity_percent=cast_element_float(subelement.attrib['percentage'])
                val_ethincity_name=subelement.text
                

                return{
                    'country':ielement.find('name').text,
                    'totalPopulation': val_population,
                    'ethincity_name':val_ethincity_name,
                    'ethincity_percent':val_ethincity_percent,
                    'ethincity_population': (val_ethincity_percent*val_population)/100
               }
        

In [33]:
#2. 10 cities with the largest population
df_city_population = pd.DataFrame(columns={'country','city','year','population'})

for element in document_tree.iterfind('country'):
    df_city_population= df_city_population.append(find_city_population(element),ignore_index=True)
    for provelement in element.iterfind('province'):
        df_city_population= df_city_population.append(find_city_population(provelement,element.find('name').text),
                                                      ignore_index=True)
        
df_city_population.sort_values(by='population',ascending=False).head(10)

,year,city,country,population
589,2010,Shanghai,China,22315474
410,2012,Istanbul,Turkey,13710512
691,2011,Mumbai,India,12442373
237,2013,Moskva,Russia,11979529
588,2010,Beijing,China,11716620
1230,2010,São Paulo,Brazil,11152344
590,2010,Tianjin,China,11090314
564,2010,Guangzhou,China,11071424
708,2011,Delhi,India,11034555
570,2010,Wuhan,China,9785388


In [34]:
#3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
# print names of all countries and their cities
#2. 10 cities with the largest population
df_ethinic_population = pd.DataFrame(columns={'country','totalPopulation','ethincity_name',
                                              'ethincity_percent','ethincity_population'})

for element in document_tree.iterfind('country'):
    df_ethinic_population= df_ethinic_population.append(find_ethinic_population(element),ignore_index=True)

    
#df_ethinic_population.sort_values(by='ethincity_population',ascending=False).head(10)
df_ethinic_population_f=pd.DataFrame(df_ethinic_population.groupby('ethincity_name').sum())
df_ethinic_population_f.sort_values(by='ethincity_population',ascending=False).head(10)

,ethincity_population,ethincity_percent,totalPopulation
ethincity_name,,,
Han Chinese,1.245059e+09,91.50,1.360720e+09
European,4.410033e+08,645.92,8.163821e+08
Dravidian,3.027137e+08,25.00,1.210855e+09
African,1.986050e+08,1055.60,2.186632e+08
Bengali,1.467769e+08,98.00,1.497724e+08
Mestizo,1.419729e+08,783.70,2.251272e+08
Japanese,1.265342e+08,99.40,1.272980e+08
Russian,1.146462e+08,79.80,1.436669e+08
Javanese,1.134560e+08,45.00,2.521245e+08


In [35]:
#4.name and country of a) longest river, b) largest lake and c) airport at highest elevation

4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

Approach 
1. Already have data from the file. 
2. iterate over the airport and river node and build the data in pandas series
3. Created methods for reusablity & avoidindg exception
4. The airport and river has country code . So extracted data from country node to build country and code
5. Built data frame from further processing using pandas. 
6. The two data frame are merged later to get the country from code

In [36]:
# built a data frame for c
df_country = pd.DataFrame(columns={'countrycode','name'})

for element in document_tree.iterfind('country'):
    if is_attrib_in_element(element,'car_code'):
        df_country=df_country.append({
           'country':element.find('name').text,
           'countrycode': element.attrib['car_code']
        },ignore_index=True)

In [37]:
# Method to send element and propert name to get data frame for that. Can be used to send aiport node and elevation as child node
def find_highest(ielement,propertynames):
    df_op = pd.DataFrame(columns={'countrycode',propertynames})
    
    for subelement in document_tree.iterfind(ielement):
        if is_attrib_in_element(subelement,'country'):
            country_val=subelement.attrib['country']
            if(' ' in country_val):
                country_val=country_val.partition(' ')[0] 
            elevation=0
            for pinfo in subelement.iterfind(propertynames):
                newelevation =cast_element_float(pinfo.text)
                #print(pinfo.text)
                if elevation < newelevation:
                    elevation =newelevation
        df_op=df_op.append({
            'countrycode':country_val,
            'name':subelement.find('name').text,
            propertynames:elevation
            
       },ignore_index=True)
    
    return df_op

In [38]:
# fidning highest airport 
df_hightestairport=find_highest('airport','elevation')
df_hightestairport_country= df_hightestairport.merge(df_country, on='countrycode')
df_hightestairport_country.sort_values('elevation',ascending=False).head(1)

,elevation,countrycode,name_x,name_y,country
80,4063.0,BOL,El Alto Intl,NaN,Bolivia


In [39]:
#  a) longest river,
df_river=find_highest('river','length')
df_river_country= df_river.merge(df_country, on='countrycode')
df_river_country.sort_values('length',ascending=False).head(1)


,length,countrycode,name_x,name_y,country
174,6448.0,CO,Amazonas,NaN,Colombia


In [40]:
#b) largest lake
df_largestlake=find_highest('lake','area')
df_largestlake_country= df_largestlake.merge(df_country, on='countrycode')
df_largestlake_country.sort_values('area',ascending=False).head(1)

,area,countrycode,name_x,name_y,country
47,386400.0,R,Caspian Sea,NaN,Russia
